### Import Libraries


In [1]:
import pandas as pd
import numpy as np
from datetime import datetime as dt
import re
import sqlite3

### Loading the data


In [2]:
df_branch_service = pd.read_json("branch_service_transaction_info.json")
df_customer_transaction = pd.read_json("customer_transaction_info.json")

# Data Profiling


### Checking the data high level


In [3]:
df_customer_transaction.head(10)

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ORTIZ,EDUARDO,1990-07-08
1,TXN-14642,2026-05-26,NIENOW,LEA,2000-11-26
2,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
3,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
4,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
5,TXN-60295,2006-09-25,LESCH,FLETA,1993-05-22
6,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
7,TXN-40462,2021-08-21,KUHN,TOD,2002-11-25
8,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10
9,TXN-08102,2010-04-03,JOHNSON,MILTON,2003-07-10


In [4]:
df_branch_service.head(10)

,txn_id,branch_name,service,price
0,TXN-24546,MallOfAsia,Manicure,NaN
1,TXN-14642,Starmall,HairColor,NaN
2,TXN-60295,SmallMall,FootSpa,NaN
3,TXN-60295,Starmall,FootSpa,NaN
4,TXN-60295,MayMall,FootSpa,NaN
5,TXN-60295,FrankMall,FootSpa,NaN
6,TXN-40462,Starmall,HairColor,NaN
7,TXN-40462,Megamall,HairColor,NaN
8,TXN-08102,RobinsonsMall,HairColor,NaN
9,TXN-08102,SmallMall,HairColor,NaN


### Checking the dataset shape


In [5]:
print(df_customer_transaction.shape)
print(df_branch_service.shape)

(130653, 5)
(130653, 4)


### Checking the unique Transaction IDs


In [6]:
print(df_customer_transaction['txn_id'].nunique())
print(df_branch_service['txn_id'].nunique())

62354
62354


### Checking the services


In [7]:
df_branch_service['service'].unique()

array(['Manicure', 'HairColor', 'FootSpa', 'Rebond', 'Haircut',
       'NailColor', 'Pedicure'], dtype=object)

### Checking the branch names


In [8]:
df_branch_service['branch_name'].unique()

array(['MallOfAsia', 'Starmall', 'SmallMall', 'MayMall', 'FrankMall',
       'Megamall', 'RobinsonsMall', '', None, 'N/A'], dtype=object)

### Checking the prices


In [9]:
df_branch_service['price'].describe()

count    110653.000000
mean         95.966969
std         115.874117
min           0.000000
25%          30.123790
50%          66.123457
75%          88.093930
max         400.231230
Name: price, dtype: float64

### Checking the availment dates


In [10]:
print(df_customer_transaction['avail_date'].min())
print(df_customer_transaction['avail_date'].max())

2005-01-01
2030-12-30


### Checking the birthdays


In [11]:
print(df_customer_transaction['birthday'].min())
print(df_customer_transaction['birthday'].max())

1990-01-01
2010-12-30


### Checking for first name and last name


In [12]:
pattern = re.compile(r'^[a-zA-Z\s]+$')

invalid_last_names = df_customer_transaction[~df_customer_transaction['last_name'].str.match(
    pattern)]
print(invalid_last_names)

          txn_id  avail_date        last_name       first_name    birthday
60000  TXN-04581  2019-09-03     Will,,,,,,,,  Aurelio........  1993-07-12
60001  TXN-17309  2015-01-04  Krajcik,,,,,,,,    Ebony........  2003-11-18
60002  TXN-18201  2021-04-28  Gutmann,,,,,,,,  Agustin........  2009-03-11
60003  TXN-60631  2012-03-13   Kemmer,,,,,,,,     Alda........  2004-08-18
60004  TXN-60631  2012-03-13   Kemmer,,,,,,,,     Alda........  2004-08-18
...          ...         ...              ...              ...         ...
79995  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79996  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79997  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79998  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79999  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16

[20000 rows x 5 columns]


In [13]:
invalid_first_names = df_customer_transaction[~df_customer_transaction['first_name'].str.match(
    pattern)]
print(invalid_first_names)

          txn_id  avail_date        last_name       first_name    birthday
60000  TXN-04581  2019-09-03     Will,,,,,,,,  Aurelio........  1993-07-12
60001  TXN-17309  2015-01-04  Krajcik,,,,,,,,    Ebony........  2003-11-18
60002  TXN-18201  2021-04-28  Gutmann,,,,,,,,  Agustin........  2009-03-11
60003  TXN-60631  2012-03-13   Kemmer,,,,,,,,     Alda........  2004-08-18
60004  TXN-60631  2012-03-13   Kemmer,,,,,,,,     Alda........  2004-08-18
...          ...         ...              ...              ...         ...
79995  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79996  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79997  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79998  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16
79999  TXN-54391  2028-10-31    Johns,,,,,,,,   Turner........  2010-04-16

[20000 rows x 5 columns]


# Data Cleaning


### **Cleaning customer transaction**


#### Fix first name and last name format


Lowercase first name and last name


In [14]:
df_customer_transaction['first_name'] = df_customer_transaction['first_name'].str.lower()
df_customer_transaction['last_name'] = df_customer_transaction['last_name'].str.lower()
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ortiz,eduardo,1990-07-08
1,TXN-14642,2026-05-26,nienow,lea,2000-11-26
2,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
3,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
4,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,cummings,henry,2005-08-14
130649,TXN-60822,2020-09-28,feil,jermey,2010-06-15
130650,TXN-60822,2020-09-28,feil,jermey,2010-06-15
130651,TXN-01784,2015-01-11,schmidt,emilie,1996-05-21


Remove special characters


In [15]:
df_customer_transaction['last_name'] = df_customer_transaction['last_name'].str.replace(
    '\W', '', regex=True)
df_customer_transaction['first_name'] = df_customer_transaction['first_name'].str.replace(
    '\W', '', regex=True)
df_customer_transaction

<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\W'
<>:2: SyntaxWarning: invalid escape sequence '\W'
<>:4: SyntaxWarning: invalid escape sequence '\W'
C:\Users\RHODA\AppData\Local\Temp\ipykernel_13864\1912723004.py:2: SyntaxWarning: invalid escape sequence '\W'
  '\W', '', regex=True)
C:\Users\RHODA\AppData\Local\Temp\ipykernel_13864\1912723004.py:4: SyntaxWarning: invalid escape sequence '\W'
  '\W', '', regex=True)


,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,ortiz,eduardo,1990-07-08
1,TXN-14642,2026-05-26,nienow,lea,2000-11-26
2,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
3,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
4,TXN-60295,2006-09-25,lesch,fleta,1993-05-22
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,cummings,henry,2005-08-14
130649,TXN-60822,2020-09-28,feil,jermey,2010-06-15
130650,TXN-60822,2020-09-28,feil,jermey,2010-06-15
130651,TXN-01784,2015-01-11,schmidt,emilie,1996-05-21


Capitalize names


In [16]:
df_customer_transaction['first_name'] = df_customer_transaction['first_name'].str.title()
df_customer_transaction['last_name'] = df_customer_transaction['last_name'].str.title()
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,Ortiz,Eduardo,1990-07-08
1,TXN-14642,2026-05-26,Nienow,Lea,2000-11-26
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
3,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
4,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,Cummings,Henry,2005-08-14
130649,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130650,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130651,TXN-01784,2015-01-11,Schmidt,Emilie,1996-05-21


Check for remaining invalid names


In [17]:
invalid_last_names = df_customer_transaction[~df_customer_transaction['last_name'].str.match(
    pattern)]
print(invalid_last_names)

Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday]
Index: []


In [18]:
invalid_first_names = df_customer_transaction[~df_customer_transaction['first_name'].str.match(
    pattern)]
print(invalid_first_names)

Empty DataFrame
Columns: [txn_id, avail_date, last_name, first_name, birthday]
Index: []


### Remove invalid dates


Set the date to today


In [19]:
now = dt.now()

Convert availment date and birthday to datetime


In [20]:
df_customer_transaction['avail_date'] = pd.to_datetime(
    df_customer_transaction['avail_date'])
df_customer_transaction['birthday'] = pd.to_datetime(
    df_customer_transaction['birthday'])
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
0,TXN-24546,2030-09-08,Ortiz,Eduardo,1990-07-08
1,TXN-14642,2026-05-26,Nienow,Lea,2000-11-26
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
3,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
4,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,Cummings,Henry,2005-08-14
130649,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130650,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130651,TXN-01784,2015-01-11,Schmidt,Emilie,1996-05-21


Filter rows so the values in availment date and birthday are less than or equal to the current date


In [21]:
df_customer_transaction = df_customer_transaction[df_customer_transaction['avail_date'] <= now]
df_customer_transaction = df_customer_transaction[df_customer_transaction['birthday'] <= now]
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
3,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
4,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
5,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
6,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,Cummings,Henry,2005-08-14
130649,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130650,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130651,TXN-01784,2015-01-11,Schmidt,Emilie,1996-05-21


Check for the latest date in availment date and birthday


In [22]:
print(df_customer_transaction['avail_date'].max())
print(df_customer_transaction['birthday'].max())

2023-11-24 00:00:00
2010-12-30 00:00:00


Remove rows with availment date earlier than birthday


In [23]:
df_customer_transaction = df_customer_transaction[df_customer_transaction['birthday']
                                                  <= df_customer_transaction['avail_date']]
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
3,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
4,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
5,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
6,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25
...,...,...,...,...,...
130648,TXN-65468,2012-06-16,Cummings,Henry,2005-08-14
130649,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130650,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15
130651,TXN-01784,2015-01-11,Schmidt,Emilie,1996-05-21


### Remove duplicates


In [24]:
df_customer_transaction = df_customer_transaction.drop_duplicates()
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
6,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25
8,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10
12,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20
24,TXN-59988,2021-02-01,Bashirian,Brice,2004-07-20
...,...,...,...,...,...
130637,TXN-17937,2016-01-04,Torphy,Garret,1998-11-26
130646,TXN-43876,2021-11-13,Jewess,Lucio,1996-07-05
130647,TXN-65468,2012-06-16,Cummings,Henry,2005-08-14
130649,TXN-60822,2020-09-28,Feil,Jermey,2010-06-15


### Remove duplicate transaction id


In [25]:
df_customer_transaction = df_customer_transaction.drop_duplicates(subset=[
                                                                  'txn_id'])
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
6,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25
8,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10
12,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20
24,TXN-59988,2021-02-01,Bashirian,Brice,2004-07-20
...,...,...,...,...,...
130618,TXN-43035,2022-05-28,Crona,Velma,1999-06-16
130631,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21
130632,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09
130635,TXN-35117,2018-03-18,Anderson,Duncan,2010-09-12


### Remove null values


In [26]:
df_customer_transaction = df_customer_transaction.dropna()
df_customer_transaction

,txn_id,avail_date,last_name,first_name,birthday
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22
6,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25
8,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10
12,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20
24,TXN-59988,2021-02-01,Bashirian,Brice,2004-07-20
...,...,...,...,...,...
130618,TXN-43035,2022-05-28,Crona,Velma,1999-06-16
130631,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21
130632,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09
130635,TXN-35117,2018-03-18,Anderson,Duncan,2010-09-12


### **Cleaning branch service transaction**


### Fix branch name and service format


Add space in between words


In [27]:
df_branch_service['branch_name'] = df_branch_service['branch_name'].apply(
    lambda x: re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x) if x is not None else x)
df_branch_service['service'] = df_branch_service['service'].apply(
    lambda x: re.sub(r'(?<=[a-z])(?=[A-Z])', ' ', x) if x is not None else x)
df_branch_service

,txn_id,branch_name,service,price
0,TXN-24546,Mall Of Asia,Manicure,NaN
1,TXN-14642,Starmall,Hair Color,NaN
2,TXN-60295,Small Mall,Foot Spa,NaN
3,TXN-60295,Starmall,Foot Spa,NaN
4,TXN-60295,May Mall,Foot Spa,NaN
...,...,...,...,...
130648,TXN-65468,Mall Of Asia,Haircut,66.123457
130649,TXN-60822,Mall Of Asia,Rebond,400.231230
130650,TXN-60822,Frank Mall,Rebond,400.231230
130651,TXN-01784,Robinsons Mall,Hair Color,88.093930


Convert all null equivalent values to None


In [28]:
df_branch_service['branch_name'] = df_branch_service['branch_name'].replace(
    ['None', 'N/A', 'NA', ''], [None, None, None, None])
df_branch_service

,txn_id,branch_name,service,price
0,TXN-24546,Mall Of Asia,Manicure,NaN
1,TXN-14642,Starmall,Hair Color,NaN
2,TXN-60295,Small Mall,Foot Spa,NaN
3,TXN-60295,Starmall,Foot Spa,NaN
4,TXN-60295,May Mall,Foot Spa,NaN
...,...,...,...,...
130648,TXN-65468,Mall Of Asia,Haircut,66.123457
130649,TXN-60822,Mall Of Asia,Rebond,400.231230
130650,TXN-60822,Frank Mall,Rebond,400.231230
130651,TXN-01784,Robinsons Mall,Hair Color,88.093930


Drop null values in branch name


In [29]:
df_branch_service = df_branch_service.dropna(subset=['branch_name'])
df_branch_service

,txn_id,branch_name,service,price
0,TXN-24546,Mall Of Asia,Manicure,NaN
1,TXN-14642,Starmall,Hair Color,NaN
2,TXN-60295,Small Mall,Foot Spa,NaN
3,TXN-60295,Starmall,Foot Spa,NaN
4,TXN-60295,May Mall,Foot Spa,NaN
...,...,...,...,...
130648,TXN-65468,Mall Of Asia,Haircut,66.123457
130649,TXN-60822,Mall Of Asia,Rebond,400.231230
130650,TXN-60822,Frank Mall,Rebond,400.231230
130651,TXN-01784,Robinsons Mall,Hair Color,88.093930


Check the values of branch name and service


In [30]:
df_branch_service['branch_name'].unique()

array(['Mall Of Asia', 'Starmall', 'Small Mall', 'May Mall', 'Frank Mall',
       'Megamall', 'Robinsons Mall'], dtype=object)

In [31]:
df_branch_service['service'].unique()

array(['Manicure', 'Hair Color', 'Foot Spa', 'Rebond', 'Nail Color',
       'Haircut', 'Pedicure'], dtype=object)

### Fix price format


Round off to 2 decimal places


In [32]:
df_branch_service['price'] = df_branch_service['price'].round(2)
df_branch_service['price']

C:\Users\RHODA\AppData\Local\Temp\ipykernel_13864\1380487666.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_branch_service['price'] = df_branch_service['price'].round(2)


0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
           ...  
130648     66.12
130649    400.23
130650    400.23
130651     88.09
130652     88.09
Name: price, Length: 116054, dtype: float64

Drop rows of price with null values


In [33]:
df_branch_service = df_branch_service.dropna(subset=['price'])
df_branch_service

,txn_id,branch_name,service,price
20000,TXN-05451,May Mall,Hair Color,0.00
20001,TXN-05451,Starmall,Hair Color,0.00
20002,TXN-02255,Frank Mall,Rebond,0.00
20003,TXN-02255,Megamall,Rebond,0.00
20004,TXN-02255,May Mall,Rebond,0.00
...,...,...,...,...
130648,TXN-65468,Mall Of Asia,Haircut,66.12
130649,TXN-60822,Mall Of Asia,Rebond,400.23
130650,TXN-60822,Frank Mall,Rebond,400.23
130651,TXN-01784,Robinsons Mall,Hair Color,88.09


Drop rows of price with 0 value


In [34]:
df_branch_service = df_branch_service[df_branch_service['price'] > 0]
df_branch_service

,txn_id,branch_name,service,price
40000,TXN-49846,Mall Of Asia,Nail Color,30.12
40001,TXN-49846,May Mall,Nail Color,30.12
40002,TXN-09756,Robinsons Mall,Haircut,66.12
40003,TXN-09756,Small Mall,Haircut,66.12
40004,TXN-09756,Small Mall,Haircut,66.12
...,...,...,...,...
130648,TXN-65468,Mall Of Asia,Haircut,66.12
130649,TXN-60822,Mall Of Asia,Rebond,400.23
130650,TXN-60822,Frank Mall,Rebond,400.23
130651,TXN-01784,Robinsons Mall,Hair Color,88.09


Check unique combinations of service and price


In [35]:
print(df_branch_service.loc[(df_branch_service['price'] >= 0), [
      'service', 'price']].drop_duplicates())

          service   price
40000  Nail Color   30.12
40002     Haircut   66.12
40005    Pedicure   77.99
40008    Foot Spa  100.12
40010      Rebond  400.23
40029    Manicure   55.23
40056  Hair Color   88.09


### Drop duplicate rows


In [36]:
print(df_branch_service.shape)
df_branch_service = df_branch_service.drop_duplicates()
print(df_branch_service.shape)

(80593, 4)
(72036, 4)


### **Merge customer_transaction and branch_service**


In [37]:
df_merged = pd.merge(df_customer_transaction, df_branch_service)
df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Small Mall,Foot Spa,100.12
1,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Starmall,Foot Spa,100.12
2,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,May Mall,Foot Spa,100.12
3,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Frank Mall,Foot Spa,100.12
4,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25,Starmall,Hair Color,88.09
...,...,...,...,...,...,...,...,...
53080,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21,Mall Of Asia,Nail Color,30.12
53081,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09,Starmall,Hair Color,88.09
53082,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09,Frank Mall,Hair Color,88.09
53083,TXN-35117,2018-03-18,Anderson,Duncan,2010-09-12,Starmall,Nail Color,30.12


Drop duplicate transaction IDs


In [38]:
df_merged = df_merged.drop_duplicates(subset='txn_id')
df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Small Mall,Foot Spa,100.12
4,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25,Starmall,Hair Color,88.09
6,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10,Robinsons Mall,Hair Color,88.09
10,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20,Small Mall,Rebond,400.23
12,TXN-08158,2023-08-05,Batz,Timmothy,2008-10-06,May Mall,Nail Color,30.12
...,...,...,...,...,...,...,...,...
53077,TXN-15004,2019-10-03,Kiehn,Nicole,2002-05-05,May Mall,Haircut,66.12
53078,TXN-40346,2012-04-06,Wintheiser,Kathryn,1994-03-03,Frank Mall,Manicure,55.23
53080,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21,Mall Of Asia,Nail Color,30.12
53081,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09,Starmall,Hair Color,88.09


Reset index


In [39]:
df_merged = df_merged.reset_index(drop=True)
df_merged

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price
0,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Small Mall,Foot Spa,100.12
1,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25,Starmall,Hair Color,88.09
2,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10,Robinsons Mall,Hair Color,88.09
3,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20,Small Mall,Rebond,400.23
4,TXN-08158,2023-08-05,Batz,Timmothy,2008-10-06,May Mall,Nail Color,30.12
...,...,...,...,...,...,...,...,...
29379,TXN-15004,2019-10-03,Kiehn,Nicole,2002-05-05,May Mall,Haircut,66.12
29380,TXN-40346,2012-04-06,Wintheiser,Kathryn,1994-03-03,Frank Mall,Manicure,55.23
29381,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21,Mall Of Asia,Nail Color,30.12
29382,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09,Starmall,Hair Color,88.09


# Data Validation


### Check the youngest customer that availed


In [40]:
df_merged['birthday'] = pd.to_datetime(df_merged['birthday'])
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'])

df_merged['age'] = np.floor(
    (df_merged['avail_date'] - df_merged['birthday']).dt.days / 365.25)

df_age_data = df_merged[['txn_id', 'avail_date', 'last_name',
                         'first_name', 'birthday', 'age', 'branch_name', 'service', 'price']]

print(df_age_data.loc[df_age_data['age'].idxmin()])

txn_id                   TXN-52584
avail_date     2006-06-05 00:00:00
last_name                  Carroll
first_name                 Bethany
birthday       2005-06-13 00:00:00
age                            0.0
branch_name             Frank Mall
service                   Foot Spa
price                       100.12
Name: 49, dtype: object


In [41]:
df_merged['avail_date'] = pd.to_datetime(df_merged['avail_date'])
print(df_merged.groupby([df_merged['avail_date'].dt.to_period(
    "W-Mon"), 'branch_name'])['price'].sum().to_frame())

                                       price
avail_date            branch_name           
2004-12-28/2005-01-03 Mall Of Asia    430.35
                      Megamall         55.23
                      Robinsons Mall  118.21
2005-01-04/2005-01-10 Frank Mall      874.82
                      Mall Of Asia     88.09
...                                      ...
2023-11-21/2023-11-27 Mall Of Asia    188.21
                      May Mall         30.12
                      Robinsons Mall  108.11
                      Small Mall      584.56
                      Starmall        698.90

[6807 rows x 1 columns]


<h1> Ingesting Data to Database </h1>


In [42]:
conn = sqlite3.connect(
    r'C:\Users\RHODA\LabExercise3\Data-Pipelines-Exercise\database\merged.db')

In [43]:
create_sql = "CREATE TABLE IF NOT EXISTS 'transaction' (txn_id TEXT, avail_date DATE, last_name TEXT, first_name TEXT, birthday DATE, branch_name TEXT, service TEXT, price INTEGER, age INTEGER)"
cursor = conn.cursor()
cursor.execute(create_sql)

In [44]:
for row in df_merged.itertuples():
    row_avail_date = row[2].strftime('%d-%m-%Y')
    row_birthday = row[5].strftime('%d-%m-%Y')
    insert_sql = f"INSERT INTO 'transaction' (txn_id, avail_date, last_name, first_name, birthday, branch_name, service, price, age) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)"
    cursor.execute(insert_sql, (row[1], row_avail_date, row[3],
                   row[4], row_birthday, row[6], row[7], row[8], row[9]))

In [45]:
conn.commit()

In [177]:
sample = pd.read_parquet(
    r"D:\Github\School Projects\Data Warehousing Class\LabExercise3\parquet\merged\add_age.parquet")
sample

,txn_id,avail_date,last_name,first_name,birthday,branch_name,service,price,age
0,TXN-60295,2006-09-25,Lesch,Fleta,1993-05-22,Small Mall,Foot Spa,100.12,13
1,TXN-40462,2021-08-21,Kuhn,Tod,2002-11-25,Starmall,Hair Color,88.09,18
2,TXN-08102,2010-04-03,Johnson,Milton,2003-07-10,Robinsons Mall,Hair Color,88.09,6
3,TXN-64262,2018-11-23,Hackett,Maci,2005-06-20,Small Mall,Rebond,400.23,13
4,TXN-08158,2023-08-05,Batz,Timmothy,2008-10-06,May Mall,Nail Color,30.12,14
...,...,...,...,...,...,...,...,...,...
29379,TXN-15004,2019-10-03,Kiehn,Nicole,2002-05-05,May Mall,Haircut,66.12,17
29380,TXN-40346,2012-04-06,Wintheiser,Kathryn,1994-03-03,Frank Mall,Manicure,55.23,18
29381,TXN-59520,2015-07-18,Kessler,Russell,1991-08-21,Mall Of Asia,Nail Color,30.12,23
29382,TXN-08837,2014-04-28,Hettinger,Carlie,1990-12-09,Starmall,Hair Color,88.09,23
